In [7]:
import pandas as pd
import spacy
from collections import defaultdict

# Carregar o modelo de português do spaCy
nlp = spacy.load('pt_core_news_sm')


In [10]:


# Texto de teste
texto = ("Em 02 de setembro de 2024 Ilmo Sr. LEONILDO ROLIM DOS SANTOS Ref: Parecer Jurídico Sigiloso (10/06/2024) e Proposta (16/08/2024)A/C.: Dr. Luis Bitetti – OAB/SP nº 84.009E-mail: bitetti@adv.oabsp.org.br ; Prezado Senhor,	Na qualidade de advogados da BRL TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A., cessionária dos honorários contratuais então titulados pelo escritório AFFONSO FERREIRA ADVOGADOS, Dra. HALBA MERY PEREBONI ROCCO e Dr. HENRIQUE LINDENBOJM, em resposta ao parecer e à proposta em referência, e considerando que:(a) a ora Notificante adquiriu de boa-fé os créditos de honorários devidos por décadas de patrocínio da ação de Desapropriação Indireta nº 0527173-63.1994.4.03.6100, cujos requisitórios foram expedidos de forma apartada da verba principal;  b) Tal verba, de natureza alimentar (Súmula Vinculante 47/STF), é autônoma  e não possui relação com a indenização devida pela União em decorrência da desapropriação de áreas situadas no município de Ubatuba/SP (Ubatumirim) na década de setenta para a expansão de rodovia federal (BR-101); (c) Eventual confusão entre áreas desapropriadas, conforme alegação descrita no parecer, se daria exclusivamente por uma sobreposição de glebas dos Autores com outra limítrofe e titulada pela companhia AGRÍCOLA AREIA BRANCA. Aliás, conforme expressamente reconhecido na proposta de 16/08/2024; e (d) O transcurso de décadas sem oposição, a tempo e modo, pelo Notificado (Sr. Leonildo) ocorrendo, sem qualquer dúvida, a extinção de eventual direito pela ocorrência da prescrição, uma vez que a o trecho da estrada BR-101 reivindicado fora construído há mais de 50 (cinquenta) anos.Vimos notificá-lo da inconsistência da pretensão vertida no parecer e na proposição e da tentativa ilegítima da postulação frente à Notificante, a qual defenderá seu direito em qualquer via que se fizer necessária, inclusive na seara deontológica.")
teste = ("Em 02 de setembro de 2024, o Sr. Afranio Affonso Ferreira Neto solicitou que a empresa BRL Trust Distribuidora de Títulos e Valores Mobiliários S.A. fosse notificada no processo judicial nº 0527173-63.1994.4.03.6100. A correspondência foi enviada para o endereço Rua Hungria nº 888 - 5º andar, Jardim Europa, São Paulo - 01455-000, e um e-mail de confirmação foi encaminhado para advocacia@affonsoferreira.com.br. Para mais informações, o advogado responsável, registrado sob a OAB/SP nº 155.406, pode ser contatado pelo telefone 55 11 3813-9522.")


# Dividir o texto em linhas (cada frase como uma linha)
linhas = texto.split('; ')

# Inicializar a lista para armazenar as linhas da tabela
tabela = []


In [11]:
for linha_num, linha in enumerate(linhas, start=1):
    doc = nlp(linha)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    
    # Inicializar dicionário para a linha
    linha_dict = {'Linha': linha_num}
    
    # Contadores para duplicar colunas se necessário
    contadores = defaultdict(int)
    
    for ent_text, ent_label in entidades:
        contadores[ent_label] += 1
        label_col = f"{ent_label}_{contadores[ent_label]}" if contadores[ent_label] > 1 else ent_label
        linha_dict[label_col] = ent_text
    
    # Adicionar a coluna 'TEXT' com o conteúdo restante
    # Remover as entidades do texto
    texto_sem_entidades = linha
    for ent_text, ent_label in entidades:
        texto_sem_entidades = texto_sem_entidades.replace(ent_text, '')
    
    linha_dict['TEXT'] = texto_sem_entidades.strip()
    
    tabela.append(linha_dict)

In [12]:
# Criar o DataFrame
df_tabela = pd.DataFrame(tabela)

pd.set_option('display.max_columns', None)
# Configurar para exibir todas as linhas (cuidado com DataFrames muito grandes)
pd.set_option('display.max_rows', None)
# Configurar a largura máxima de cada coluna
pd.set_option('display.max_colwidth', None)

print(df_tabela)

   Linha                        PER                    PER_2  \
0      1  Parecer Jurídico Sigiloso                 Proposta   
1      2         FERREIRA ADVOGADOS  Desapropriação Indireta   
2      3                        NaN                      NaN   
3      4                    Autores                      NaN   
4      5               Sr. Leonildo                      NaN   

                MISC             LOC                     LOC_2  \
0            C.: Dr.              SP  bitetti@adv.oabsp.org.br   
1                BRL  Prezado Senhor                    que:(a   
2  Súmula Vinculante             STF                     União   
3                NaN             NaN                       NaN   
4                NaN          BR-101               Notificante   

                                                                                                                                                                                                                          

In [13]:
import openpyxl
df_tabela.to_excel('tabela-texto-teste.xlsx', index=False)